<a href="https://colab.research.google.com/github/MarinaEstefania/data-engineering-bootcamp/blob/main/Notebooks/dim_tables.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.2.2/spark-3.2.2-bin-hadoop3.2.tgz
!tar xf spark-3.2.2-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark

import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.2.2-bin-hadoop3.2"

import findspark
findspark.init()
findspark.find()

from pyspark.sql import DataFrame, SparkSession, Window
from pyspark.sql.types import StructType,StructField, StringType, IntegerType
from pyspark.sql.functions import *
from pyspark.sql.window import *

spark = SparkSession \
       .builder \
       .appName("review_logs") \
       .config('spark.jars.packages', 'com.databricks:spark-xml_2.12:0.15.0')\
       .getOrCreate()

spark

In [2]:
#Extract data
df = spark.read.parquet("sample_data/part-00000-c3039b56-7ffc-4dae-b3da-aaa9da7784c8-c000.snappy.parquet")
df.show()

+----------+--------+-------------+-----------------+------------+------------+--------------+------+
|  log_date|  device|     location|               os|          ip|phone_number|       browser|log_id|
+----------+--------+-------------+-----------------+------------+------------+--------------+------+
|04-25-2021|  Mobile|       Kansas|        Apple iOS|9.200.232.57|821-540-5777|        Safari|     1|
|03-13-2021|  Tablet|       Oregon|   Google Android|9.200.232.57|819-102-1320| Google Chrome|     2|
|09-30-2021|  Tablet|    Minnesota|        Apple iOS|9.200.232.57|989-156-0498|        Safari|     3|
|05-24-2021|  Tablet|     Arkansas|      Apple MacOS|9.200.232.57|225-837-9935|        Safari|     4|
|02-01-2021|  Tablet|New Hampshire|            Linux|9.200.232.57|243-842-4562|       Firefox|     5|
|07-23-2021|  Tablet|  Pensylvania|        Apple iOS|9.200.232.57|694-501-4352|        Safari|     6|
|10-13-2021|Computer|     New York|      Apple MacOS|9.200.232.57|430-449-7136|   

In [7]:
#Transform data (create dim dataframes)
dim_devices = df.select(col('log_id'),col('device')) \
  .drop_duplicates(['device']) \
  .withColumnRenamed('log_id', 'id_dim_devices')
dim_devices.show()

dim_location = df.select(col('log_id'),col('location')) \
  .drop_duplicates(['location']) \
  .withColumnRenamed('log_id', 'id_dim_location')
dim_location.show()

dim_os = df.select(col('log_id'),col('os')) \
  .drop_duplicates(['os']) \
  .withColumnRenamed('log_id', 'id_dim_os')
dim_os.show()

dim_browser = df.select(col('log_id'),col('browser')) \
  .drop_duplicates(['browser']) \
  .withColumnRenamed('log_id', 'id_dim_browser')
dim_browser.show()

dim_date = df.select(df.log_id, df.log_date) \
  .drop_duplicates(['log_date']) \
  .withColumnRenamed('log_id', 'id_dim_date')
dim_date = dim_date.withColumn('day', split(col('log_date'),'-').getItem(1)) \
  .withColumn('month', split(col('log_date'),'-').getItem(0)) \
  .withColumn('year', split(col('log_date'),'-').getItem(2)) \
  .withColumn('season', lit('under construction...'))
dim_date.show(5)

+--------------+--------+
|id_dim_devices|  device|
+--------------+--------+
|             7|Computer|
|             1|  Mobile|
|             2|  Tablet|
+--------------+--------+

+---------------+-------------+
|id_dim_location|     location|
+---------------+-------------+
|             55|         Utah|
|             52|       Hawaii|
|              3|    Minnesota|
|             31|         Ohio|
|              2|       Oregon|
|              4|     Arkansas|
|             24|        Texas|
|             90| North Dakota|
|             89|  Connecticut|
|             17|     Nebraska|
|            148|      Vermont|
|            114|       Nevada|
|              9|   Washington|
|             33|     Illinois|
|            107|     Oklahoma|
|             63|     Delaware|
|            177|       Alaska|
|             39|   New Mexico|
|             25|West Virginia|
|             19|     Missouri|
+---------------+-------------+
only showing top 20 rows

+---------+------------

In [4]:
#Load data
#Load data from AWS Glue Studio to Snowflake DataWarehouse
dim_devices.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions).option("dbtable", "DIM_DEVICES").mode("overwrite").save()
dim_location.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions).option("dbtable", "DIM_LOCATION").mode("overwrite").save()
dim_os.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions).option("dbtable", "DIM_OS").mode("overwrite").save()
dim_browser.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions).option("dbtable", "DIM_BROWSER").mode("overwrite").save()
dim_date.write.format(SNOWFLAKE_SOURCE_NAME).options(**sfOptions).option("dbtable", "DIM_DATE").mode("overwrite").save()

job.commit()
#fact_movie_analytics.write.option("header", "true").csv("s3://manual-bucket-megc/gold-data/dim_devices/")



Py4JJavaError: ignored